In [1]:
import os

In [4]:
%pwd

'/Users/sidharthsingh/Desktop/chicken_disease_AWS'

In [3]:
os.chdir("../")

In [5]:
# Update Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
## Config Manager

class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [12]:
## Update Components
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-14 21:50:34,788]: INFO: common: yaml file: config/config.yaml Loaded Successfully]
[2023-06-14 21:50:34,789]: INFO: common: yaml file: params.yaml Loaded Successfully]
[2023-06-14 21:50:34,790]: INFO: common: created directory at: artifacts]
[2023-06-14 21:50:34,791]: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-14 21:50:37,756]: INFO: 1668759883: artifacts/data_ingestion/data.zip downloaded with following info: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E600:1F3F:280C1B:3A95F3:6489E8D3
Accept-Ranges: bytes
Date: Wed, 14 Jun 2023 16:20:36 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830023-CCU
X-Cac